In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn import svm

from supervised_sentiment_analysis import *
from constants import *
from utilities import *

In [2]:
merged_results = pickle.load(open('merged_results.P', 'rb'))
merged_results['Valid Vector'] = merged_results['Skip Thought Vector'].apply(lambda x: ~np.isnan(x).any())
removed_results = merged_results[~merged_results['Valid Vector']]
merged_results = merged_results[merged_results['Valid Vector']]

In [3]:
labeled_results = merged_results[merged_results['Categorical Tag'] != 'no tag']
q1_results = merged_results[merged_results['Question'] == Q1]
q1_labeled_results = labeled_results[labeled_results['Question'] == Q1]
q1_features = np.array(q1_labeled_results['Skip Thought Vector'].tolist())
q1_labels = np.array(q1_labeled_results['Categorical Tag'])

In [4]:
clf = svm.SVC()
clf.fit(q1_features, q1_labels)
predictions = clf.predict(list(q1_results['Skip Thought Vector']))
q1_results['prediction'] = predictions

/yw_data/robert_gold/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
def get_average_sentiment(results):
    y_pred = list(results['prediction'])
    total = max(len(y_pred), 1)
    positive_count = len([prediction for prediction in y_pred if prediction == 'positive'])
    positive_score = positive_count/total
    negative_count = len([prediction for prediction in y_pred if prediction == 'negative'])
    negative_score = negative_count/total
    
    return [positive_score, negative_score, 1-positive_score-negative_score, positive_score-negative_score]
get_average_sentiment(q1_results)

[0.23526448362720404,
 0.2397984886649874,
 0.5249370277078086,
 -0.004534005037783367]

In [6]:
def get_sentiment_scores(results):
    problem_keys = {
        'fex1': 0,
        'fex2': 1,
        'fex4': 2,
        'ps1': 3,
        'ps2': 4,
        'ps4': 5,
    }
    results['problem key'] = results['Problem'].apply(lambda x: problem_keys[x])
    subsets = [[k] for k in range(6)]+[[0,3],[1,4],[2,5],[0,1,2],[3,4,5], list(range(6))]
    sentiment_scores = []
    for subset in subsets:
        subset_results = results[results['problem key'].isin(subset)]
        sentiment_scores.append(get_average_sentiment(subset_results))

    arr = np.array(sentiment_scores)
    
    index = ['fex1', 'fex2', 'fex4', 'ps1', 'ps2', 'ps4', '1', '2', '4', 'fex', 'ps', 'total']
    columns = ['Positive', 'Negative', 'Neutral', 'Sentiment']
    rounded = np.round(arr, 2)
    return pd.DataFrame(rounded, columns=columns, index=index)

In [ ]:
get_sentiment_scores(q1_results)

/yw_data/robert_gold/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Positive,Negative,Neutral,Sentiment
fex1,0.20,0.28,0.52,-0.09
fex2,0.19,0.30,0.50,-0.11
fex4,0.27,0.15,0.58,0.12
ps1,0.22,0.22,0.56,0.00
ps2,0.25,0.25,0.50,0.01
ps4,0.29,0.25,0.47,0.04
1,0.21,0.24,0.55,-0.03
2,0.22,0.28,0.50,-0.05
4,0.28,0.19,0.54,0.09
fex,0.22,0.24,0.53,-0.02
